In [1]:
#import funcs
import funcs

In [2]:
#use recipe string and number of search pages as arguements
#return two pandas DataFrames (top recipes info/recipe text(ingredients and instructions))
top_recipes_df, recipe_text_df = funcs.recipe_search('coconut cake',2)

### Needs debugging removed ###

-------------------
Starting scrape for page 1
https://www.allrecipes.com/search?q=coconut+cake
-------------------
5 elements searched of 24
10 elements searched of 24
15 elements searched of 24
20 elements searched of 24
-------------------
Moving to next page
-------------------
15 recipes found on page 1
-------------------
Starting scrape for page 2
https://www.allrecipes.com/search?coconut%20cake=coconut%20cake&offset=24&q=coconut%20cake
-------------------
5 elements searched of 24
10 elements searched of 24
15 elements searched of 24
20 elements searched of 24
-------------------
19 recipes found on page 2
-------------------
Reached the last page...
All recipes scraped and stored...
34 total recipes found
-------------------


In [15]:
#print top_recipes df here
top_recipes_df = top_recipes_df.reset_index(drop = True)
top_recipes_df

,title,rating,url
0,Creamy Coconut Cake,5,https://www.allrecipes.com/recipe/23762/creamy...
1,Coconut Coconut Milk Cake,5,https://www.allrecipes.com/recipe/237023/cocon...
2,Toasted Almond and Coconut Cake with White Cho...,5,https://www.allrecipes.com/recipe/8532961/toas...
3,Old Fashioned Coconut Cake,5,https://www.allrecipes.com/recipe/274082/old-f...
4,Simply the Best Coconut Cake,5,https://www.allrecipes.com/recipe/284066/simpl...
5,Coconut Milk Cake Mix Cake,5,https://www.allrecipes.com/recipe/244620/cocon...
6,Coconut Rum Cake,5,https://www.allrecipes.com/recipe/284300/cocon...
7,Coconut Coffee Liqueur Cake,5,https://www.allrecipes.com/recipe/8363/coconut...


In [29]:
# USE THIS CELL TO PULL SPECIFIC RECIPE TITLE

#enter row index for row_index variable
row_index = 2
recipe_title = top_recipes_df.iloc[row_index,0]

In [28]:
# copy & paste title from df above in quotes
# or use recipe_title
recipe_choice = recipe_title

# takes a string, the recipe_text_df and top_recipes_df as arguements
# prints ingredients and instructions to screen
funcs.get_recipe(recipe_choice,recipe_text_df, top_recipes_df)

Url: 
https://www.allrecipes.com/recipe/274082/old-fashioned-coconut-cake/

----------------------
Ingredients:
----------------------
½ cup butter, room temperature
½ cup margarine, room temperature
2 cups white sugar
3 cups cake flour
1 tablespoon baking powder
½ teaspoon salt
4  eggs, room temperature
1 cup unsweetened coconut milk
2 teaspoons vanilla extract
1 cup sour cream
¾ cup white sugar
½ cup sweetened flaked coconut
4 tablespoons milk
1 cup white sugar
5  egg whites
1 tablespoon vanilla extract
⅛ teaspoon salt
1 ½ cups unsalted butter
2 tablespoons sweetened flaked coconut, or as desired


Step 1:
----------------------
Preheat the oven to 350 degrees F (175 degrees C). Grease and flour three 9-inch cake pans.
----------------------

Step 2:
----------------------
Beat butter and margarine together using an electric mixer in a bowl until creamy. Add sugar gradually, beating until light and fluffy.
----------------------

Step 3:
----------------------
Sift flour, baking powd